In [1]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 30.0 MB/s eta 0:00:0000:0100:01


In [2]:
topics = []
with open("./inputs/topics.txt", "r") as file:
    topics = file.read().split()
topics

['Sports-and-Outdoors',
 'Home-and-Kitchen',
 'Grocery-and-Gourmet-Food',
 'Automotive',
 'Software',
 'Office-Products',
 'Clothing-Shoes-and-Jewelry',
 'Baby',
 'Cell-Phones-and-Accessories',
 'Patio-Lawn-and-Garden',
 'Beauty',
 'Musical-Instruments',
 'Health-and-Personal-Care',
 'Pet-Supplies',
 'Electronics',
 'Tools-and-Home-Improvement',
 'Toys-and-Games',
 'Industrial-and-Scientific',
 'Video-Games',
 'Arts-Crafts-and-Sewing',
 'Appliances']

In [3]:
from collections import defaultdict
from psycopg2 import sql
import psycopg2

In [4]:
def get_column_names(conn, table):
    """
    Fetches and prints the column names of a specified table.
    """
    query = sql.SQL("""
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name = %s
        ORDER BY ordinal_position;
    """)
    cur = conn.cursor()
    cur.execute(query, (table,))
    columns = cur.fetchall()
    cur.close()
    return [col[0] for col in columns]

In [8]:
dbname = "db"  # internal database name
username = "user"
password = "password"

table = "test"  # Name of table to query
count_topic = defaultdict(int)

for topic in topics:
    conn = psycopg2.connect(
                dbname=dbname,
                user=username,
                password=password,
                host=topic,
            )

    # Prepare and execute the query
    query = sql.SQL("SELECT id, question, answer, keywords FROM {}").format(sql.Identifier(table))
    try:
        with conn.cursor() as cur:
            cur.execute(query)
            rows = cur.fetchall()
            count_topic[topic] = len(rows)
    except psycopg2.errors.UndefinedTable:  # Handle table not found error
        print(f"Table '{table}' does not exist in database '{dbname}' on node '{topic}'")
        count_topic[topic] = 0  # Or handle the case as you see fit
    finally:
        conn.close()

# sort by desending frequency
count_topic = dict(sorted(count_topic.items(), key=lambda x: x[1], reverse=True))

print('Num Topic:', len(count_topic))
count_topic

Num Topic: 21


{'Electronics': 314263,
 'Home-and-Kitchen': 184439,
 'Sports-and-Outdoors': 146891,
 'Tools-and-Home-Improvement': 101088,
 'Automotive': 89923,
 'Cell-Phones-and-Accessories': 85865,
 'Health-and-Personal-Care': 80496,
 'Patio-Lawn-and-Garden': 59595,
 'Toys-and-Games': 51486,
 'Office-Products': 43608,
 'Beauty': 42422,
 'Pet-Supplies': 36607,
 'Baby': 28933,
 'Musical-Instruments': 23322,
 'Clothing-Shoes-and-Jewelry': 22068,
 'Arts-Crafts-and-Sewing': 21262,
 'Grocery-and-Gourmet-Food': 19538,
 'Video-Games': 13307,
 'Industrial-and-Scientific': 12136,
 'Software': 10636,
 'Appliances': 9011}

In [14]:
dbname = "db"  # internal database name
username = "user"
password = "password"

table = "test"  # Name of table to query
count_topic = defaultdict(int)

# test for topic[0]
topic = topics[2]
conn = psycopg2.connect(
        dbname=dbname,
        user=username,
        password=password,
        host=topic,
    )

query = sql.SQL("SELECT id, question, answer, keywords FROM {}").format(sql.Identifier(table))

try:
    with conn.cursor() as cur:
        cur.execute(query)
        rows = cur.fetchall()
except psycopg2.errors.UndefinedTable:  # Handle table not found error
    print(f"Table '{table}' does not exist in database '{dbname}' on node '{topic}'")
finally:
    conn.close()

for row in rows[10000:11000]:
    print(row)


(9981, 'what is in the dessicant packet?', 'Silica Gel. PACKET - Fast and extra adsorbent silica gel. Meets FDA specifications for food and drug products. Provides superior adsorption. Remains dry at maximum saturation. Silica gel can adsorb 15% of its weight in water vapor in 2 hours.', ['kw1', 'kw2', 'kw3', 'kw4', 'kw5'])
(9982, 'Are these snack from Japan? Or eastern United States? Thanks', 'Package says they are Product of Korea.', ['kw1', 'kw2', 'kw3', 'kw4', 'kw5'])
(9983, 'Sodium content? Could someone post the nutrition label for this?', "2 servings per container. Amt. per serving calories is 30 total fax 2g saturated & trans fat 0g Sodium 50mg Carb 1g Dietery Fiber 1g Protein 1g. Had a 5 star review up there, but I no longer stand by that since I came across roasted seaweed snacks by Gimme Roasted Seaweed Snacks. Those are USDA certified Organic seaweed, organic sesame & organic sunflower oil & sea salt. I am much more comfortable with that brand's organic ingredients & person